Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Constants

In [ ]:
from config import DATASET_PATH,LABEL_PATH

# Define paths and constants
dataset_directory = DATASET_PATH
labels_path = LABEL_PATH
batch_size = 32
epochs = 10
IMG_SIZE = (160, 160)
CLASSES = ['cooling', 'pollen', 'varroa', 'wasps', 'bee']


Unprocessed data

In [ ]:
# Load labels from JSON
import json

with open(labels_path) as json_file:
    labels = json.load(json_file)
    
# Create a list of image paths and corresponding labels
image_paths = []
image_labels = []

for image_name, label in labels.items():
    image_path = os.path.join(dataset_directory, image_name)
    image_paths.append(image_path)
    image_labels.append(label)


# Processed data

In [ ]:
labels = []
for label_dict in image_labels:
    label = [lbl for lbl, has in label_dict.items() if has]
    if 'wasps' not in label:
        label.append('bee')
    labels.append(label)

train_data = pd.DataFrame({'image_paths': image_paths, 'labels': labels})

In [ ]:
train_data.shape

Split test dataset

In [ ]:
test_data = train_data.sample(frac=0.02)
train_data = train_data.drop(test_data.index)

In [ ]:
print(train_data.shape)
print(test_data.shape)

Data generation

In [ ]:
# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.7, 1.3),
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1,
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input
)

In [ ]:
# Data augmentation with generator
train_generator = datagen.flow_from_dataframe(
    train_data,
    x_col='image_paths',
    y_col='labels',
    target_size=IMG_SIZE,
    batch_size=batch_size,
    classes=CLASSES,
    class_mode='categorical',
    shuffle=True
)


# Model

In [ ]:

# Load MobileNet model with pre-trained weights and exclude the top layer
base_model = tf.keras.applications.MobileNet(
    input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet'
)

base_model.trainable = False

# Add custom classification layers
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='sigmoid'),
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_data.shape[0] // batch_size,
    epochs=epochs
)


In [ ]:
# Evaluate the model on the test dataset
test_generator = datagen.flow_from_dataframe(
    test_data,
    x_col='image_paths',
    y_col='labels',
    target_size=IMG_SIZE,
    batch_size=batch_size,
    classes=CLASSES,
    class_mode='categorical',
    shuffle=False
)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Calculate and display metrics
print(classification_report(true_classes, predicted_classes))
confusion_mtx = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(confusion_mtx)


In [ ]:
model.save('bee_classification_model.h5')
